# Your info

Full name:Niki Nezakati

Student ID:98522094

## Q4

In [13]:
# Q4_graded
# Do not change the above line.

import random

population = []

def initialize_population(pop_size=1000):
    global population
    population = []
    for i in range(pop_size):
        population.append(random.uniform(-10, 10))

def fitness(x):
    return abs(168 * x ** 3 - 7.22 * x ** 2 + 15.5 * x - 13.2)


def sort_by_fitness():
    global population
    population.sort(key=fitness)


def reproduce(parent1, parent2, range=10):
    midpoint = random.randrange(1, range)
    child1 = (parent1 * midpoint + parent2 * (range - midpoint)) / range
    child2 = (parent2 * midpoint + parent1 * (range - midpoint)) / range
    return child1, child2

def crossover(c_range=10):
    global population
    for i in range(len(population) // 2):
        x = population[2 * i]
        y = population[2 * i + 1]
        child1, child2 = reproduce(x, y, c_range)
        population[2 * i] = child1
        population[2 * i + 1] = child2


def mutate(alpha):
    global population
    for i in range(len(population)):
        if random.random() < alpha:
            population[i] += random.uniform(-0.1, 0.1)


def GA(pop_size=1000, threshold=0.000001, crossover_range=10, alpha=0.1):
    initialize_population(pop_size)
    sort_by_fitness()
    generation = 0
    while fitness(population[0]) > threshold:
        crossover(crossover_range)
        mutate(alpha)
        sort_by_fitness()
        generation += 1
    return population[0]



print(GA())    



0.3692877567453548


## Q5


In [14]:
# Q5_graded
# Do not change the above line.

class Task:
  def __init__(self, time_demand: float, machine_demand: float):
    self.time_demand = time_demand
    self.machine_demand = machine_demand

class Machine:
  def __init__(self, machine_id: int, time_supply: float, time_velocity: float, machine_supply: float, machine_capacity: float):
    self.id = machine_id
    self.time_supply = time_supply
    self.time_velocity = time_velocity
    self.machine_supply = machine_supply
    self.machine_capacity = machine_capacity      

In [15]:
# Q5_graded
# Do not change the above line.
import numpy as np
from typing import List
from matplotlib import pyplot as plt

class ACOscheduler:
  def __init__(self, tasks, machines, population_number=100, iterations=500):
    self.tasks = tasks
    self.machines = machines
    self.task_num = len(tasks) # The number of tasks  
    self.machine_number = len(machines) # Number of machines 
    self.population_number = population_number # Population number 
    self.iterations = iterations 
    # The pheromone of the machine representing the task selection 
    self.pheromone_phs = [[100 for _ in range(self.machine_number)] for _ in range(self.task_num)]
    self.best_pheromone = None

# Generate a new solution vector 
  def gen_pheromone_jobs(self):
    ans = [-1 for _ in range(self.task_num)]
    node_free = [node_id for node_id in range(self.machine_number)]
    for let in range(self.task_num):
      ph_sum = np.sum(list(map(lambda j: self.pheromone_phs[let][j], node_free)))
      test_val = 0
      rand_ph = np.random.uniform(0, ph_sum)
      for node_id in node_free:
        test_val += self.pheromone_phs[let][node_id]
        if rand_ph <= test_val:
          ans[let] = node_id
          break
    return ans

# Evaluate the current solution vector 
  def evaluate_particle(self, pheromone_jobs: List[int]) -> int:
    time_util = np.zeros(self.machine_number)
    machine_util = np.zeros(self.machine_number)

    for i in range(len(self.machines)):
      time_util[i] = self.machines[i].time_supply
      machine_util[i] = self.machines[i].machine_supply

    for i in range(self.task_num):
      time_util[pheromone_jobs[i]] += self.tasks[i].time_demand
      machine_util[pheromone_jobs[i]] += self.tasks[i].machine_demand

    for i in range(self.machine_number):
      if time_util[i] > self.machines[i].time_velocity:
        return 100
      if machine_util[i] > self.machines[i].machine_capacity:
        return 100
    for i in range(self.machine_number):
      time_util[i] /= self.machines[i].time_velocity
      machine_util[i] /= self.machines[i].machine_capacity

    return np.std(time_util, ddof=1) + np.std(machine_util, ddof=1)


  # Calculate Fitness 
  def calculate_fitness(self, pheromone_jobs: List[int]) -> float:
    return 1 / self.evaluate_particle(pheromone_jobs)

  # Update pheromones 
  def update_pheromones(self):

    for i in range(self.task_num):
      for j in range(self.machine_number):
        if j == self.best_pheromone[i]:
          self.pheromone_phs[i][j] *= 2
        else:
          self.pheromone_phs[i][j] *= 0.5

  def scheduler_main(self):
    results = [0 for _ in range(self.iterations)]
    fitness = 0

    for it in range(self.iterations):
      best_time = 0
      for ant_id in range(self.population_number):
        pheromone_jobs = self.gen_pheromone_jobs()
        fitness = self.calculate_fitness(pheromone_jobs)
        if fitness > best_time:
          self.best_pheromone = pheromone_jobs
          best_time = fitness
          assert self.best_pheromone is not None
          self.update_pheromones()
          results[it] = best_time
          if it % 10 == 0:
            print("ACO iter: ", it, " / ", self.iterations, ", Fitness : ", fitness)
    return results


if __name__ == '__main__':
  nodes = [Machine(0, 0.862, 950, 950, 1719), Machine(1, 0.962, 2, 950, 1719), Machine(2, 1.062, 2, 1500, 1719)]
  lets = [Task(0.15, 50), Task(0.05, 100), Task(0.2, 60),
  Task(0.01, 70), Task(0.04, 80), Task(0.07, 20),
  Task(0.14, 150), Task(0.15, 200), Task(0.03, 40), Task(0.06, 90)]
  ac = ACOscheduler(lets, nodes, iterations=150)
  res = ac.scheduler_main()
  i = 0
  for _ in ac.best_pheromone:
    print(" Task :", i, " Put it on the machine ", ac.best_pheromone[i])
    i += 1

ACO iter:  0  /  150 , Fitness :  0.01
ACO iter:  0  /  150 , Fitness :  1.9757380856930447
ACO iter:  0  /  150 , Fitness :  1.983983586616108
ACO iter:  0  /  150 , Fitness :  2.179601865260971
ACO iter:  10  /  150 , Fitness :  1.983983586616108
ACO iter:  20  /  150 , Fitness :  1.983983586616108
ACO iter:  30  /  150 , Fitness :  1.983983586616108
ACO iter:  40  /  150 , Fitness :  1.983983586616108
ACO iter:  50  /  150 , Fitness :  1.983983586616108
ACO iter:  60  /  150 , Fitness :  1.983983586616108
ACO iter:  70  /  150 , Fitness :  1.983983586616108
ACO iter:  80  /  150 , Fitness :  1.983983586616108
ACO iter:  90  /  150 , Fitness :  1.983983586616108
ACO iter:  100  /  150 , Fitness :  1.983983586616108
ACO iter:  110  /  150 , Fitness :  1.983983586616108
ACO iter:  120  /  150 , Fitness :  1.983983586616108
ACO iter:  130  /  150 , Fitness :  1.983983586616108
ACO iter:  140  /  150 , Fitness :  1.983983586616108
 Task : 0  Put it on the machine  1
 Task : 1  Put it on 

# <font color='red'>Submission</font>

1. Sign up in [Gradescope](https://www.gradescope.com) with proper name and student ID.
2. Fill in your full name (seperated by single spaces) and student ID in the beginning of this notebook.
3. After you're done with this notebook, you should do the following:
  - Clear all outputs of the notebook.
  ![clear all outputs](https://i.ibb.co/y6FrttB/Screen-Shot-2021-03-21-at-01-51-42.png)
  - Run all of the cells (if you skipped a question just leave the cell unchanged), and make sure all of your outputs are correct.
  ![run all](https://i.ibb.co/cgRcBZ0/Screen-Shot-2021-03-21-at-01-54-58.png)
  - Save your notebook.
  
  - If you're using Colab, download your notebook.
  ![download ipynb](https://i.ibb.co/2KxYM6K/Screen-Shot-2021-03-21-at-02-03-50.png)
  
  - Put the notebook file you just downloaded and `convert.py` in the same folder run the following command:
  ```bash
  python convert.py
  ```
  This will export your code for each question into a `.py` file.
    - **Note**: if you want to add more cells, add this to the **first** line of the cell:
  ```python
  # Q4_graded or #Q5_graded
  ```
  according to the question number.
  - There are 2 assignments in Gradescope:
  
    You should upload your **codes** and your **notebook** in `HW5` section and your final report for all of the questions as a **single pdf** file in `HW5 - Report`. Autograder will automatically check for:
    - `CI002_HW5.ipynb`
    - `Q4.py`
    - `Q5.py`
    - Your name and ID in the beginning of `.ipynb` file.

    It is important that you <font color='red'>**don't**</font> change the names of these files before submission.

4. If you pass the autograder, you're good to go.